In [1]:
import os
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime
import zipfile

In [2]:
#Ustalenie katalogu roboczego ze zmiennej środowiskowej. 
if os.getenv("Gdrive_studia") is not None:
    os.chdir(os.getenv("Gdrive_studia"))

In [3]:
# Folder docelowy na pobrane pliki
folder_docelowy = "Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/"

# Tworzy folder docelowy jeżeli nie istnieje
os.makedirs(folder_docelowy, exist_ok=True)

#### Stworzenie listy linków na bazie formatu używanego przez IMGW

In [4]:
def stworz_liste_linkow_do_pobrania(rok_poczatkowy: int,) -> list:
    baza_url = "https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/"    
    lista_linków = []
    rok = rok_poczatkowy
    while rok <2000:
        for i in range(0,5):
            download_link = f"{baza_url}{rok}_{rok+4}/{rok+i}_k.zip"
            print(download_link)
            lista_linków.append(download_link)
        rok += 5
    obecny_rok = datetime.now().year
    while rok <= obecny_rok:
        for miesiac in range(1,13):
            miesiac_sformatowany = f"{miesiac:02}"
            download_link = f"{baza_url}{rok}/{rok}_{miesiac_sformatowany}_k.zip"
            lista_linków.append(download_link)
        rok += 1
    return lista_linków
linki_do_wywolania = stworz_liste_linkow_do_pobrania(rok_poczatkowy=1951)

https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1951_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1952_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1953_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1954_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1955_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1956_1960/1956_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1956_1960/1957_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1956_1960/1958_k.zip
https://danepubliczne.imgw.pl/data/dane_pomiarow

#### Funkcja pobierająca dane

In [5]:
def pobierz_dane_z_linkow(lista_linków, folder_docelowy):
    """
    Pobiera dane z listy linków i zapisuje je w podanym folderze.

    Args:
        linki_do_wywolania (list): Lista linków do pobrania.
        folder_docelowy (str): Ścieżka do folderu, w którym mają być zapisane pliki.
    Returns:
        dict: Podsumowanie pobrań (sukcesy i błędy).
    """
    # Upewnij się, że folder docelowy istnieje
    os.makedirs(folder_docelowy, exist_ok=True)

    sukcesy = 0
    niepowodzenia = 0

    for link in linki_do_wywolania:
        try:
            # Pobierz nazwę pliku z linku
            nazwa_pliku = link.split('/')[-1]
            sciezka_do_pliku = os.path.join(folder_docelowy, nazwa_pliku)
            if nazwa_pliku not in os.listdir(folder_docelowy):
                # Pobierz dane z linku
                print(f"Pobieranie: {link}")
                response = requests.get(link, stream=True)
                response.raise_for_status()  # Sprawdź, czy nie było błędów HTTP

                # Zapisz plik na dysku
                with open(sciezka_do_pliku, 'wb') as plik:
                    for chunk in response.iter_content(chunk_size=8192):
                        plik.write(chunk)

                print(f"Pobrano i zapisano: {sciezka_do_pliku}")
                sukcesy += 1
        except requests.exceptions.RequestException as e:
            print(f"Błąd podczas pobierania {link}: {e}")
            niepowodzenia += 1

    # Zwróć podsumowanie
    return {
        "sukcesy": sukcesy,
        "niepowodzenia": niepowodzenia
    }

#### Pobranie danych

In [6]:
pobierz_dane_z_linkow(lista_linków=linki_do_wywolania ,folder_docelowy=folder_docelowy)

Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1951_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1952_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1952_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1953_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1953_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1951_1955/1954_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1954_k.zip
Pobieranie: 

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1984_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1981_1985/1985_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1985_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1986_1990/1986_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1986_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1986_1990/1987_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1987_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/1986_1990/1988_k.zip
Pobrano i za

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002_06_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002/2002_07_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002_07_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002/2002_08_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002_08_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002/2002_09_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002_09_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2002/2002_10_k.zip
Pobrano 

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005_04_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005/2005_05_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005_05_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005/2005_06_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005_06_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005/2005_07_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005_07_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2005/2005_08_k.zip
Pobrano 

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008_03_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008/2008_04_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008_04_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008/2008_05_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008_05_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008/2008_06_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008_06_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2008/2008_07_k.zip
Pobrano 

...

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2013_11_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2013/2013_12_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2013_12_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2014/2014_01_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2014_01_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2014/2014_02_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2014_02_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2014/2014_03_k.zip
Pobrano 

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2016_10_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2016/2016_11_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2016_11_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2016/2016_12_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2016_12_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2017/2017_01_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2017_01_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2017/2017_02_k.zip
Pobrano 

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019_08_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019/2019_09_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019_09_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019/2019_10_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019_10_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019/2019_11_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019_11_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2019/2019_12_k.zip
Pobrano 

Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022_06_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022/2022_07_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022_07_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022/2022_08_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022_08_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022/2022_09_k.zip
Pobrano i zapisano: Big Data/Projekt/Dane/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022_09_k.zip
Pobieranie: https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/dobowe/klimat/2022/2022_10_k.zip
Pobrano 

{'sukcesy': 336, 'niepowodzenia': 2}

#### Wypakowanie wszystkich plików zip

In [7]:
import os
import zipfile

def wypakuj_pliki_zip(folder_docelowy):
    """
    Wypakowuje wszystkie pliki ZIP w podanym folderze.

    Args:
        folder_docelowy (str): Ścieżka do folderu, w którym znajdują się pliki ZIP.
    """
    błędy = {}
    
    # Wypakuj wszystkie pliki ZIP
    for nazwa_pliku in os.listdir(folder_docelowy):
        sciezka_pliku = os.path.join(folder_docelowy, nazwa_pliku)

        # Sprawdź, czy plik jest archiwum ZIP
        if zipfile.is_zipfile(sciezka_pliku):
            try:
                print(f"Wypakowywanie: {nazwa_pliku}")
                with zipfile.ZipFile(sciezka_pliku, 'r') as zip_ref:
                    zip_ref.extractall(folder_docelowy)
                print(f"Pomyślnie wypakowano: {nazwa_pliku}")
            except Exception as e:
                błędy[nazwa_pliku] = e
                print(f"Błąd podczas wypakowywania {nazwa_pliku}: {e}")
    
    # Usuń pliki ZIP po udanym wypakowaniu     
    for nazwa_pliku in os.listdir(folder_docelowy):
        if nazwa_pliku not in list(błędy.keys()):      
            sciezka_pliku = os.path.join(folder_docelowy, nazwa_pliku)
            if sciezka_pliku.endswith('.zip') and os.path.isfile(sciezka_pliku):
                try:
                    os.remove(sciezka_pliku)
                    print(f"Usunięto plik ZIP: {nazwa_pliku}")
                except Exception as e:
                    błędy[nazwa_pliku] = e
                    print(f"Błąd podczas usuwania {nazwa_pliku}: {e}")
        else:
            continue

    return błędy


In [8]:
a = {"2": 23,"3":14}
type(a.keys())

dict_keys

In [9]:
wypakuj_pliki_zip(folder_docelowy)
#TODO
# zmergowanie funkcji tak by automatycznie po sukcesie usuwano plik zip, chyba, że był bład wtedy zwrócenie słownika z błędami

Wypakowywanie: 1951_k.zip
Pomyślnie wypakowano: 1951_k.zip
Wypakowywanie: 1952_k.zip
Pomyślnie wypakowano: 1952_k.zip
Wypakowywanie: 1953_k.zip
Pomyślnie wypakowano: 1953_k.zip
Wypakowywanie: 1954_k.zip
Pomyślnie wypakowano: 1954_k.zip
Wypakowywanie: 1955_k.zip
Pomyślnie wypakowano: 1955_k.zip
Wypakowywanie: 1956_k.zip
Pomyślnie wypakowano: 1956_k.zip
Wypakowywanie: 1957_k.zip
Pomyślnie wypakowano: 1957_k.zip
Wypakowywanie: 1958_k.zip
Pomyślnie wypakowano: 1958_k.zip
Wypakowywanie: 1959_k.zip
Pomyślnie wypakowano: 1959_k.zip
Wypakowywanie: 1960_k.zip
Pomyślnie wypakowano: 1960_k.zip
Wypakowywanie: 1961_k.zip
Pomyślnie wypakowano: 1961_k.zip
Wypakowywanie: 1962_k.zip
Pomyślnie wypakowano: 1962_k.zip
Wypakowywanie: 1963_k.zip
Pomyślnie wypakowano: 1963_k.zip
Wypakowywanie: 1964_k.zip
Pomyślnie wypakowano: 1964_k.zip
Wypakowywanie: 1965_k.zip
Pomyślnie wypakowano: 1965_k.zip
Wypakowywanie: 1966_k.zip
Pomyślnie wypakowano: 1966_k.zip
Wypakowywanie: 1967_k.zip
Pomyślnie wypakowano: 1967_k.z

Pomyślnie wypakowano: 2007_10_k.zip
Wypakowywanie: 2007_11_k.zip
Pomyślnie wypakowano: 2007_11_k.zip
Wypakowywanie: 2007_12_k.zip
Pomyślnie wypakowano: 2007_12_k.zip
Wypakowywanie: 2008_01_k.zip
Pomyślnie wypakowano: 2008_01_k.zip
Wypakowywanie: 2008_02_k.zip
Pomyślnie wypakowano: 2008_02_k.zip
Wypakowywanie: 2008_03_k.zip
Pomyślnie wypakowano: 2008_03_k.zip
Wypakowywanie: 2008_04_k.zip
Pomyślnie wypakowano: 2008_04_k.zip
Wypakowywanie: 2008_05_k.zip
Pomyślnie wypakowano: 2008_05_k.zip
Wypakowywanie: 2008_06_k.zip
Pomyślnie wypakowano: 2008_06_k.zip
Wypakowywanie: 2008_07_k.zip
Pomyślnie wypakowano: 2008_07_k.zip
Wypakowywanie: 2008_08_k.zip
Pomyślnie wypakowano: 2008_08_k.zip
Wypakowywanie: 2008_09_k.zip
Pomyślnie wypakowano: 2008_09_k.zip
Wypakowywanie: 2008_10_k.zip
Pomyślnie wypakowano: 2008_10_k.zip
Wypakowywanie: 2008_11_k.zip
Pomyślnie wypakowano: 2008_11_k.zip
Wypakowywanie: 2008_12_k.zip
Pomyślnie wypakowano: 2008_12_k.zip
Wypakowywanie: 2009_01_k.zip
Pomyślnie wypakowano: 200

Wypakowywanie: 2018_06_k.zip
Pomyślnie wypakowano: 2018_06_k.zip
Wypakowywanie: 2018_07_k.zip
Pomyślnie wypakowano: 2018_07_k.zip
Wypakowywanie: 2018_08_k.zip
Pomyślnie wypakowano: 2018_08_k.zip
Wypakowywanie: 2018_09_k.zip
Pomyślnie wypakowano: 2018_09_k.zip
Wypakowywanie: 2018_10_k.zip
Pomyślnie wypakowano: 2018_10_k.zip
Wypakowywanie: 2018_11_k.zip
Pomyślnie wypakowano: 2018_11_k.zip
Wypakowywanie: 2018_12_k.zip
Pomyślnie wypakowano: 2018_12_k.zip
Wypakowywanie: 2019_01_k.zip
Pomyślnie wypakowano: 2019_01_k.zip
Wypakowywanie: 2019_02_k.zip
Pomyślnie wypakowano: 2019_02_k.zip
Wypakowywanie: 2019_03_k.zip
Pomyślnie wypakowano: 2019_03_k.zip
Wypakowywanie: 2019_04_k.zip
Pomyślnie wypakowano: 2019_04_k.zip
Wypakowywanie: 2019_05_k.zip
Pomyślnie wypakowano: 2019_05_k.zip
Wypakowywanie: 2019_06_k.zip
Pomyślnie wypakowano: 2019_06_k.zip
Wypakowywanie: 2019_07_k.zip
Pomyślnie wypakowano: 2019_07_k.zip
Wypakowywanie: 2019_08_k.zip
Pomyślnie wypakowano: 2019_08_k.zip
Wypakowywanie: 2019_09_k.

Usunięto plik ZIP: 2010_07_k.zip
Usunięto plik ZIP: 2010_08_k.zip
Usunięto plik ZIP: 2010_09_k.zip
Usunięto plik ZIP: 2010_10_k.zip
Usunięto plik ZIP: 2010_11_k.zip
Usunięto plik ZIP: 2010_12_k.zip
Usunięto plik ZIP: 2011_01_k.zip
Usunięto plik ZIP: 2011_02_k.zip
Usunięto plik ZIP: 2011_03_k.zip
Usunięto plik ZIP: 2011_04_k.zip
Usunięto plik ZIP: 2011_05_k.zip
Usunięto plik ZIP: 2011_06_k.zip
Usunięto plik ZIP: 2011_07_k.zip
Usunięto plik ZIP: 2011_08_k.zip
Usunięto plik ZIP: 2011_09_k.zip
Usunięto plik ZIP: 2011_10_k.zip
Usunięto plik ZIP: 2011_11_k.zip
Usunięto plik ZIP: 2011_12_k.zip
Usunięto plik ZIP: 2012_01_k.zip
Usunięto plik ZIP: 2012_02_k.zip
Usunięto plik ZIP: 2012_03_k.zip
Usunięto plik ZIP: 2012_04_k.zip
Usunięto plik ZIP: 2012_05_k.zip
Usunięto plik ZIP: 2012_06_k.zip
Usunięto plik ZIP: 2012_07_k.zip
Usunięto plik ZIP: 2012_08_k.zip
Usunięto plik ZIP: 2012_09_k.zip
Usunięto plik ZIP: 2012_10_k.zip
Usunięto plik ZIP: 2012_11_k.zip
Usunięto plik ZIP: 2012_12_k.zip
Usunięto p

{'2024_07_k.zip': zipfile.BadZipFile("Bad CRC-32 for file 'k_d_07_2024.csv'")}